# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [30]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier

## Read the CSV and Perform Basic Data Cleaning

In [12]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

le = LabelEncoder()

le.fit(df["home_ownership"])
df["home_ownership"] = le.transform(df["home_ownership"])

le.fit(df["verification_status"])
df["verification_status"] = le.transform(df["verification_status"])

le.fit(df["issue_d"])
df["issue_d"] = le.transform(df["issue_d"])

le.fit(df["pymnt_plan"])
df["pymnt_plan"] = le.transform(df["pymnt_plan"])

le.fit(df["initial_list_status"])
df["initial_list_status"] = le.transform(df["initial_list_status"])

le.fit(df["next_pymnt_d"])
df["next_pymnt_d"] = le.transform(df["next_pymnt_d"])

le.fit(df["application_type"])
df["application_type"] = le.transform(df["application_type"])

le.fit(df["hardship_flag"])
df["hardship_flag"] = le.transform(df["hardship_flag"])

le.fit(df["debt_settlement_flag"])
df["debt_settlement_flag"] = le.transform(df["debt_settlement_flag"])

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,2,low_risk,0,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,2,low_risk,0,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,2,low_risk,0,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,2,low_risk,0,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,2,low_risk,0,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


## Split the Data into Training and Testing

In [13]:
# Create our features
X = df.copy()
X.drop("loan_status", axis=1, inplace=True)
X.head()

# Create our target
y = df["loan_status"]

In [14]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.0,68817.000000,68817.000000,...,68817.000000,68817.000000,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.805542,0.0,21.778153,0.217766,...,95.057627,30.626217,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.714932,0.0,20.199244,0.718367,...,8.326426,33.631463,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.000000,0.0,0.000000,0.000000,...,20.000000,0.000000,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.000000,0.0,13.890000,0.000000,...,93.000000,0.000000,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,1.000000,0.0,19.760000,0.000000,...,100.000000,20.000000,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,1.000000,0.0,26.660000,0.000000,...,100.000000,50.000000,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,2.000000,0.0,999.000000,18.000000,...,100.000000,100.000000,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0


In [15]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [21]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [22]:
# Create the StandardScaler instance
scaler = StandardScaler()

In [23]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [24]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [25]:
# Resample the training data with the BalancedRandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=1)
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [26]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, predictions)

0.678100890070298

In [27]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   36,    65],
       [    4, 17100]])

In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.90      0.36      1.00      0.51      0.60      0.33       101
   low_risk       1.00      1.00      0.36      1.00      0.60      0.38     17104

avg / total       1.00      1.00      0.36      1.00      0.60      0.38     17205



In [29]:
# List the features sorted in descending order by feature importance
importances_sorted = sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.06889544695030737, 'total_pymnt'),
 (0.06809326619937071, 'total_rec_int'),
 (0.06111818224033979, 'total_rec_prncp'),
 (0.06013301494612743, 'last_pymnt_amnt'),
 (0.05692471219840015, 'total_pymnt_inv'),
 (0.019042688224441683, 'dti'),
 (0.0179398573175023, 'installment'),
 (0.017710744043574745, 'out_prncp_inv'),
 (0.017429948763759796, 'out_prncp'),
 (0.017376997366953917, 'max_bal_bc')]

### Easy Ensemble Classifier

In [31]:
# Train the Classifier
learning_rates = [0.05, 0.1, 0.25, 0.5, 0.75, 1]
for learning_rate in learning_rates:
    model = GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=learning_rate,
        max_features=2,
        max_depth=3,
        random_state=0)
    model.fit(X_train_scaled,y_train.ravel())
    print("Learning rate: ", learning_rate)

    print("Accuracy score (training): {0:.3f}".format(
        model.score(
            X_train_scaled,
            y_train.ravel())))
    print("Accuracy score (validation): {0:.3f}".format(
        model.score(
            X_test_scaled,
            y_test.ravel())))
    print()

Learning rate:  0.05
Accuracy score (training): 0.996
Accuracy score (validation): 0.995

Learning rate:  0.1
Accuracy score (training): 0.997
Accuracy score (validation): 0.995

Learning rate:  0.25
Accuracy score (training): 0.997
Accuracy score (validation): 0.994

Learning rate:  0.5
Accuracy score (training): 0.990
Accuracy score (validation): 0.990

Learning rate:  0.75
Accuracy score (training): 0.850
Accuracy score (validation): 0.853

Learning rate:  1
Accuracy score (training): 0.119
Accuracy score (validation): 0.120



In [32]:
model = GradientBoostingClassifier(
    n_estimators=500,
    learning_rate=.1,
    max_features=5,
    max_depth=3,
    random_state=0)

# Fit the model
model.fit(X_train_scaled,y_train.ravel())


GradientBoostingClassifier(max_features=5, n_estimators=500, random_state=0)

In [33]:
# Calculated the balanced accuracy score
predictions = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, predictions)

0.587734963276496

In [34]:
# Display the confusion matrix
confusion_matrix(y_test, predictions)

array([[   18,    83],
       [   47, 17057]])

In [35]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.28      0.18      1.00      0.22      0.42      0.16       101
   low_risk       1.00      1.00      0.18      1.00      0.42      0.19     17104

avg / total       0.99      0.99      0.18      0.99      0.42      0.19     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    Balanced Random Forest.

2. Which model had the best recall score?

    Balanced Random Forest.

3. Which model had the best geometric mean score?

    Balanced Random Forest.

4. What are the top three features?

    total_pymnt, total_rec_int, total_rec_prncp